# Part 1/2 creating random data and transorm it

*This project is made for Piotr Mitkowski portfolio to show some usage of data analyst python libraries to create, transform, and visual represent alphanumerical data*
###### Imagine there a company that sells consoles, games and accesories via Internet in Central Easter Europe (Mostly in Poland, excluding Russia..). As Polish company all sales is calculated into "PLN" currency, Foreign sales is booked always in EUR. Data are randomly generated but distribution is set with some indicators eg. to make Polish sales highest compared with other countries
*Bullet Points of part 1/2- used skills*
* *1.working with pandas (creating new columns using .apply / .map / operands/ np.select/ custom functions)*
* *2.dummy data generator using faker*
* *3.Using API to get EUR/PLN fx*
* *4.importing / exporting data to excel*
* *5.export generated sales data to .json for further analysis*

# imports

In [1]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None  # default='warn'

import random # as the data is all invented it will be mostly based on this library
from faker import Faker ##to create some dummy data

from pathlib import Path
import matplotlib.pyplot as plt

#to time some of my operations
from timeit import default_timer as timer

#progress bar 
from tqdm.auto import tqdm

#rquest to update NBP fx rates viaAPI
import requests

from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

### date setting 

In [2]:
now = datetime.now()
current_date = now.strftime("%Y-%m-%d")
current_year = now.year
current_month = now.month

### Paths

In [3]:
CWD = Path.cwd()
DATA = CWD / "DATA"
OUTPUT = CWD/ "OUTPUT_"
SETTINGS_FILE = DATA / "initial_data&settings.xlsx"
FX_RATES_FILE = DATA / "NBP_EUR_FX_ARCHIVE.xlsx"

# Data imports

In [4]:
# country/product initial settings
#editing this excel files allows you to adjust datacreated as you want, you can add new countries, products, make some them return like 50% of purchases
country_df = pd.read_excel(SETTINGS_FILE,sheet_name="Countries", index_col=0)
product_df = pd.read_excel(SETTINGS_FILE,sheet_name="Products", index_col=0)
discounts_df =pd.read_excel(SETTINGS_FILE,sheet_name="Discounts", index_col=0)

#FX rates
fx_df = pd.read_excel(FX_RATES_FILE)

#dictionaries for sales country/ sales product distribution 
sales_country_ind = country_df["ind_sales_weights"].to_dict()
sales_product_ind = product_df["ind_sales_weights"].to_dict()
sales_country_names = country_df["official_name_en"].to_dict()
country_infl_ind= country_df["ind_inflationRate"].to_dict()
discounts_rates_dict = discounts_df['discount_rate'].to_dict()

### my helpfer methods --> saved as external library

In [5]:
%%writefile DATA/helper_methods.py
from datetime import datetime, timedelta
import pandas as pd
import random
import requests

#API method to update fx
def getFXs(start_date: datetime, stop_date: datetime, currencyCode: str) -> pd.DataFrame:
    """
    get fx rates from NBP api to PLN. Unfortunately api does not allows more period than 90 days meaning stop - start < 90 days. This returns pandas DataFrame
    """
    #shift to string data
    start_date,stop_date = start_date.strftime("%Y-%m-%d"),stop_date.strftime("%Y-%m-%d")
    url_template = f"http://api.nbp.pl/api/exchangerates/rates/a/{currencyCode}/{start_date}/{stop_date}/"
    request = requests.get(url_template).json()
    df_fx = pd.json_normalize(data=request, record_path='rates')

    return df_fx

def string_date_to_date_time(string_date: str) ->datetime:
    return datetime(int(string_date[:4]), 
                               int(string_date[5:7]),
                               int(string_date[-2:]))

def last_day_of_month(date: datetime) -> datetime:
    thirty = [4,6,9,11]
    thirty_one = [1,3,5,7,8,10,12]
    if (date.month in thirty):
        return datetime(date.year, date.month, 30)
    elif (date.month in thirty_one):
        return datetime(date.year, date.month, 31)
    elif ((date.year % 4 == 0 and date.year % 100 !=0) or (date.year % 400 == 0)):
        return datetime(date.year, date.month, 29)
    else:
        return datetime(date.year, date.month, 28)
    
def random_date_day(year: int,month: int, ):
    max = last_day_of_month(datetime(year,month,1)).day
    #print(max)
    day = random.randint(1,max)
    date=datetime(year,month,day).strftime("%Y-%m-%d")
    return pd.Series([date,day])

def month_number_to_name(x: int):
    dict_month = {1:"January",2:"February",3:"March",4:"April",5:"May",6:"June",7:"July",8:"August",9:"September",10:"October",11:"November",12:"December"}
    return dict_month[x]

def random_weightbased_columnbased(df: pd.DataFrame, column: str):
    x = random.choices(df.index,weights=df[column].values)
    return x[0]

Overwriting DATA/helper_methods.py


In [6]:
#import above
from DATA.helper_methods import getFXs, string_date_to_date_time, last_day_of_month, random_date_day, month_number_to_name, random_weightbased_columnbased

### fx update via NBP API

In [7]:
last_update_date = fx_df['effectiveDate'].max()
#in datetime format
last_update_date_dt = string_date_to_date_time(last_update_date)
#update if update time is < now
if (last_update_date_dt + timedelta(days=1) < now):
    #print("checked")
    try:
        fx_df = pd.concat([fx_df,getFXs(last_update_date_dt + timedelta(days=1), now, "EUR")])
    except:
        print("No new fx data in NBA")
    #also updates the file
    else:
        fx_df.to_excel((FX_RATES_FILE), index=0)

In [8]:
#check for empty dates (weekends) and fill it with previous fx
test_start = string_date_to_date_time(fx_df['effectiveDate'].min())
test_end= string_date_to_date_time(fx_df['effectiveDate'].max())
days_range = (test_end - test_start).days
for single_date in (test_start + timedelta(n) for n in range(days_range+1)):
    #check for empty dates
    if len(fx_df[fx_df['effectiveDate']==single_date.strftime("%Y-%m-%d")])==0:
        #print("weszło")
        previous_day=single_date-timedelta(1)
        row = {"no": fx_df[fx_df['effectiveDate']==previous_day.strftime("%Y-%m-%d")]['no'].values[0],
               "effectiveDate" : single_date.strftime("%Y-%m-%d"),
               "mid": fx_df[fx_df['effectiveDate']==previous_day.strftime("%Y-%m-%d")]['mid'].values[0]}
        #adding row to df
        #fx_df = fx_df.append(row,ignore_index=True)
        row = pd.DataFrame(data=[row.values()], columns=row.keys())
        fx_df = pd.concat([fx_df,row],ignore_index=True)
    #print(fx_df['effectiveDate'] == single_date.strftime("%Y-%m-%d"))
#sort adjusted table by date
fx_df.sort_values(by=['effectiveDate'], inplace=True , ascending=True)
fx_df.to_excel((FX_RATES_FILE), index=0)

In [9]:
#making dictionary from dataframe --> easier to map fxes
fx_dict = fx_df.set_index('effectiveDate')['mid'].to_dict()

# Indicators settings

In [10]:
volume_ind = {1: 0.6, 2: 0.25, 3:0.05, 4:0.04, 5:0.03, 6:0.02, 7:0.01} #show volume distribution per order -> mostly ppl order 1 item
month_sales_ind = {1: 0.5,2: 0.8,3: 1,4: 1,5: 1.3,6: 1.3,7: 1,8: 1,9: 1.3,10: 1.5,11: 1.8,12: 2} #this are sales weights per month -> to model seasonal of sales in game industry e.g. Christmas sales etc


In [11]:
#dummy data Dictionaries to generate names, cities from each country
dummy_sets_dict = {}
for i in country_df.index:
    #print(country_df.loc[i]["faker_code"])
    dummy_sets_dict[i] = Faker(country_df.loc[i]["faker_code"])

# Data Creation

### main generating method

In [12]:
## method to generate data per one year
def create_data_per_year(year: int,
                         size: int,
                         year_count: int,
                        ) ->pd.DataFrame:
    """putting generating data process into method to make it easier to use in further loop. This requires year number, size number, and year_count(for inflation purposes) --> year_count = 0 means starting prices without inflation impact """
    df = pd.DataFrame()
    ####=====////======\\\\\\\\ COLUMS Creations========////===START===\\\\\\\\
    #Weighted randoms
    df["sales_country_code"] = random.choices(list(sales_country_ind.keys()),weights=list(sales_country_ind.values()), k=size)
    df["sales_country_name"] = df["sales_country_code"].map(sales_country_names)
    df["sales_region_name"] = df["sales_country_code"].map(country_df['region_name'])
    df["product_ID"] = random.choices(list(sales_product_ind.keys()),weights=list(sales_product_ind.values()), k=size)
    df["product_name"] = df["product_ID"].map(product_df["product_name"])
    df["product_category"] = df["product_ID"].map(product_df["main_category"])
    df["volume"] = random.choices(list(volume_ind.keys()),weights=list(volume_ind.values()), k=size)
    df['transaction_type'] = "gross"
    df["sales_currency"] = df["sales_country_code"].map(country_df["sales_currency"])

    df['year'] = year
    df['month'] = random.choices(list(month_sales_ind.keys()),weights=list(month_sales_ind.values()), k=size)
    df[['date','day']] = df['month'].apply(lambda x: random_date_day(year, x))
    df['city'] = df["sales_country_code"].apply(lambda x: dummy_sets_dict[x].city())
    df['user'] = df["sales_country_code"].apply(lambda x: dummy_sets_dict[x].user_name())
    df['email'] = df["sales_country_code"].apply(lambda x: dummy_sets_dict[x].email())

    #BASED ON Other columns
    #discounts based on month
    df['discount_code'] = df['month'].apply(lambda x: random_weightbased_columnbased(discounts_df, month_number_to_name(x)))
    df['discount_rate'] = df['discount_code'].map(discounts_rates_dict)
    df['discount_description'] = df['discount_code'].map(discounts_df['discount_description'])

    #sales_price_based --> based on currency --> includes yearly inflation rate => per product code
    #helper column
    df['_hc_inflation_impact'] = df['sales_country_code'].apply(lambda x: ((1+country_infl_ind[x])**year_count)) # to be removed
    df['sales_price_in_currency'] = round(df.apply(lambda x: product_df["starting_price_"+x.sales_currency].loc[x.product_ID], axis=1) * df['_hc_inflation_impact'],0)
    df['discounted_price_in_currency'] = round(df['sales_price_in_currency'] * (1-df['discount_rate']),2)
    df['total_value_in_currency'] = df["volume"] * df['discounted_price_in_currency']

    #values IN PLN
    df['EUR/PLN'] = df['date'].map(fx_dict)
    conditions = [(df["sales_currency"]=="EUR"),
                 (df["sales_currency"]=="PLN"),
                 ((df["sales_currency"]!="EUR") & (df["sales_currency"]!="PLN")),
                ]
    values = [round(df['total_value_in_currency']*df['EUR/PLN'],2),
             (df['total_value_in_currency']),
             "New-Currency"]
    df['total_value_in_pln'] = np.select(conditions, values).astype("float")

    #Cos & margin
    df["ttl_product_cost_pln"] = round(df["volume"] *df["product_ID"].map(product_df["starting_product_cost_PLN"]) * df['_hc_inflation_impact'],2)
    df["ttl_transportation_cost_pln"] = round(df["volume"] *df["product_ID"].map(product_df["starting_transportation_costs_PLN"]) * df['_hc_inflation_impact'],2)
    #margin
    df['ttl_margin_pln'] = df['total_value_in_pln'] - df["ttl_product_cost_pln"] - df["ttl_transportation_cost_pln"]

    ####=====////======\\\\\\\\ COLUMS Creations========////===END===\\\\\\\\

    #### RETURNS adjustment####
    list_of_indexes_with_returns = []
    for i in df.index:
        if random.random() < country_df['ind_returns_prob'].to_dict()[df['sales_country_code'].loc[i]]:
            list_of_indexes_with_returns.append(i)
    df_returns = df.loc[list_of_indexes_with_returns].copy()
    #adjustment -reverse sign + random volume
    for i in df_returns.index:
        bought_volume = df_returns['volume'].loc[i]
        return_volume = random.randint(1, bought_volume)
        #print(return_volume, df_returns['volume'].loc[i])
        #adjust values
        df_returns['volume'].loc[i] = - return_volume
        df_returns['transaction_type'].loc[i] = "returns"
        df_returns['total_value_in_currency'].loc[i] = - round(df_returns['total_value_in_currency'].loc[i] * return_volume /bought_volume,2)
        df_returns['ttl_product_cost_pln'].loc[i] = - round(df_returns['ttl_product_cost_pln'].loc[i] * return_volume /bought_volume,2)
        df_returns['ttl_transportation_cost_pln'].loc[i] = 0
        df_returns['total_value_in_pln'].loc[i] = - round(df_returns['total_value_in_pln'].loc[i] * return_volume /bought_volume,2)
        df_returns['ttl_margin_pln'].loc[i] = df_returns['total_value_in_pln'].loc[i] - df_returns['ttl_product_cost_pln'].loc[i]
    df = pd.concat([df,df_returns],ignore_index=True)
    df.sort_values(by=['date'], inplace=True , ascending=True)
    df.reset_index(inplace=True,drop=True)
    return df


### Loop through years

lets simulate company x sales:
1. throught 10 year since 2011 -> my EUR fx dictionary starts 2010-01-04 it updates to current date. generating data for future dates will cause empty fields for EUR fx
2. include yearly inflation per each country
3. include returns_rate per each country (to not make it more complex returns_date = sales_date)
4. yearly volume increase [%] set as we want [yearly_v_inc]
5. starting gross transactions size at given number [start_gross_vol]
6. monthly sales distributions is set to be increased in Q4 (holidays, presents)
7. includes diferent discounts (randomly chosen based on season)

In [13]:
def create_data_for_period(start_year: int, end_year: int, start_gross_vol: int, yearly_v_inc: float):
    df = pd.DataFrame()
    for year_count in tqdm(range(end_year-start_year+1)):
        adj_size = int(round(start_gross_vol*(1+yearly_v_inc)**year_count,0))
        currentYear = start_year + year_count
        #print(adj_size,currentYear)
        df_temp = create_data_per_year(currentYear, adj_size, year_count)
        df = pd.concat([df,df_temp],ignore_index=True)
        
    return df

### Our output dataframe 

In [14]:
START_YEAR = 2011
END_YEAR = 2020
SIZE = 10000
YEARLY_INC_RATE = 0.05

df_final= create_data_for_period(START_YEAR,END_YEAR, SIZE, YEARLY_INC_RATE)
df_final.columns

  0%|          | 0/10 [00:00<?, ?it/s]

Index(['sales_country_code', 'sales_country_name', 'sales_region_name',
       'product_ID', 'product_name', 'product_category', 'volume',
       'transaction_type', 'sales_currency', 'year', 'month', 'date', 'day',
       'city', 'user', 'email', 'discount_code', 'discount_rate',
       'discount_description', '_hc_inflation_impact',
       'sales_price_in_currency', 'discounted_price_in_currency',
       'total_value_in_currency', 'EUR/PLN', 'total_value_in_pln',
       'ttl_product_cost_pln', 'ttl_transportation_cost_pln',
       'ttl_margin_pln'],
      dtype='object')

## first view at final data

In [15]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136582 entries, 0 to 136581
Data columns (total 28 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   sales_country_code            136582 non-null  object 
 1   sales_country_name            136582 non-null  object 
 2   sales_region_name             136582 non-null  object 
 3   product_ID                    136582 non-null  object 
 4   product_name                  136582 non-null  object 
 5   product_category              136582 non-null  object 
 6   volume                        136582 non-null  int64  
 7   transaction_type              136582 non-null  object 
 8   sales_currency                136582 non-null  object 
 9   year                          136582 non-null  int64  
 10  month                         136582 non-null  int64  
 11  date                          136582 non-null  object 
 12  day                           136582 non-nul

In [16]:
starting_size_mb = round(df_final.memory_usage().sum()/1024/1024,3)
starting_size_mb

29.177

In [17]:
pd.DataFrame(df_final.nunique(),columns=["no_unique_values"])

,no_unique_values
sales_country_code,18
sales_country_name,18
sales_region_name,5
product_ID,30
product_name,30
product_category,3
volume,14
transaction_type,2
sales_currency,2
year,10


## datatransforms to make data smaller and faster in further works

In [18]:
#Categorical - clearly some of attributes might be categorical as they have only a few unique values
df_final['sales_country_code'] = df_final['sales_country_code'].astype("category")
df_final['sales_country_name'] = df_final['sales_country_name'].astype("category")
df_final['product_ID'] = df_final['product_ID'].astype("category")
df_final['product_name'] = df_final['product_name'].astype("category")
df_final['transaction_type'] = df_final['transaction_type'].astype("category")
df_final['sales_currency'] = df_final['sales_currency'].astype("category")
df_final['discount_code'] = df_final['discount_code'].astype("category")
df_final['sales_region_name'] = df_final['sales_region_name'].astype("category")
df_final['product_category'] = df_final['product_category'].astype("category")
df_final['discount_description'] = df_final['discount_description'].astype("category")

#numerical
df_final['discount_rate'] = df_final['discount_rate'].astype("float16")
df_final['year'] = df_final['year'].astype("int16")
df_final['month'] = df_final['month'].astype("int8")
df_final['volume'] = df_final['volume'].astype("int16")
df_final['_hc_inflation_impact'] = df_final['_hc_inflation_impact'].astype("float16")
df_final['sales_price_in_currency'] = df_final['sales_price_in_currency'].astype("float16")
df_final['discounted_price_in_currency'] = df_final['discounted_price_in_currency'].astype("float16")
df_final['total_value_in_currency'] = df_final['total_value_in_currency'].astype("float32")
df_final['EUR/PLN'] = df_final['EUR/PLN'].astype("float16")
df_final['total_value_in_pln'] = df_final['total_value_in_pln'].astype("float32")
df_final['ttl_product_cost_pln'] = df_final['ttl_product_cost_pln'].astype("float32")
df_final['ttl_transportation_cost_pln'] = df_final['ttl_transportation_cost_pln'].astype("float16")
df_final['ttl_margin_pln'] = df_final['ttl_margin_pln'].astype("float32")

#strings
df_final['city'] = df_final['city'].astype("string")
df_final['user'] = df_final['user'].astype("string")
df_final['email'] = df_final['email'].astype("string")

#date
df_final['date'] = pd.to_datetime(df_final['date'])

### impact on data size

In [19]:
end_size_mb = round(df_final.memory_usage().sum()/1024/1024,4)
print(f"staring data size: {starting_size_mb} MB\nafter datatype adjust size: {end_size_mb} MB")
shrink_rate= round((starting_size_mb-end_size_mb)/starting_size_mb*100,2)
print(f"that save up to: {starting_size_mb-end_size_mb:.3f} MB and around {shrink_rate} % of memory space")

staring data size: 29.177 MB
after datatype adjust size: 10.8161 MB
that save up to: 18.361 MB and around 62.93 % of memory space


# Exports

#### to CSV

In [20]:
#commented this as the .csv file weight to much and doesnt save column types
#%timeit df_final.to_csv(OUTPUT/f"SALES_DATA {current_date}.csv", index=0)

#### to excel

In [21]:
#commented this this was time consuming for larger that
#%timeit df_final.to_excel(OUTPUT/f"SALES_DATA {current_date}.xlsx", index=0)

#### to pickle

In [22]:
%timeit df_final.to_pickle(OUTPUT/f"SALES_DATA {current_date}.pickle")

77 ms ± 4.64 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [23]:
%timeit df_read = pd.read_pickle(OUTPUT/f"SALES_DATA {current_date}.pickle")
df_read = pd.read_pickle(OUTPUT/f"SALES_DATA {current_date}.pickle")

55.1 ms ± 2.06 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## quick check on imported data

In [24]:
df_read['year'].unique()

array([2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020],
      dtype=int16)

In [35]:
df_read.shape

(136582, 28)

In [25]:
df_read.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136582 entries, 0 to 136581
Data columns (total 28 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   sales_country_code            136582 non-null  category      
 1   sales_country_name            136582 non-null  category      
 2   sales_region_name             136582 non-null  category      
 3   product_ID                    136582 non-null  category      
 4   product_name                  136582 non-null  category      
 5   product_category              136582 non-null  category      
 6   volume                        136582 non-null  int16         
 7   transaction_type              136582 non-null  category      
 8   sales_currency                136582 non-null  category      
 9   year                          136582 non-null  int16         
 10  month                         136582 non-null  int8          
 11  date         

In [ ]:
condition = (df_read['sales_country_name']=="Serbia") & (df_read['year'] == 2020)
df_read[condition]['total_value_in_pln'].sum()

1698846.5

### in part 2of2 I will work with data created in this file and saved as .pickle.

# Notes

In [27]:
#random.choices(["test1","test","test4"],weights=(0.5,0.3,0.2), k=100)

In [28]:
#start_date = datetime(2010,1,1)
#stop_date = start_date  + timedelta(days=90)

#all_data_list = []
#while (start_date <now):
#    all_data_list.append(getFXs(start_date, stop_date, "EUR"))
#    start_date = stop_date + timedelta(days=1)
#    stop_date = start_date + timedelta(days=90)
#all_data_df = pd.concat(all_data_list)
#all_data_df.to_excel((FX_RATES_FILE), index=0)

In [29]:
#row = {"no": fx_df[fx_df['effectiveDate']=="2023-02-05"]['no'].values[0],
#               "effectiveDate" : "2023-02-04",
#               "mid": fx_df[fx_df['effectiveDate']=="2023-02-05"]['mid'].values[0]}
#test = pd.DataFrame(data=[row.values()], columns=row.keys())

In [30]:
#d_list = []
#for i in range(1000):
#    d_list.append(random_date_day(2021,2))
#d_list

In [31]:
#pd.DataFrame(df.value_counts('sales_country_code'), columns=['total_lines'])

In [32]:
#month = 4
#list(discounts_df[month_number_to_name(month)].values), discounts_df[month_number_to_name(month)].index.to_list()

In [33]:
#((1+df["sales_country_code"].map(country_infl_ind))**year_count)

In [34]:
#len(list_of_indexes_with_returns) , len(df), str(len(list_of_indexes_with_returns)/len(df)*100) + " %"